In [23]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np
import src.models.helpers as mh

import src.data.breathe_data as bd
import src.data.helpers as dh
from scipy.stats import norm
from scipy.stats import laplace
import pandas as pd

import itertools

In [2]:
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_conservative_smoothing_with_idx")

In [3]:
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,ecFEV1,ecFEF2575,Sex,Height,Age,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEF2575%ecFEV1,idx ecFEV1 (L),idx O2 saturation (%),idx ecFEF2575%ecFEV1,idx ecFEF25-75 % ecFEV1 (%)
0,101,2019-01-25,1.31,97,0.54,1.31,0.54,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,99.845492,41.221374,26,47,20,20
1,101,2019-01-26,1.31,98,0.57,1.31,0.57,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,100.874827,43.511450,26,48,21,21
2,101,2019-01-27,1.31,96,0.67,1.31,0.67,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,98.816157,51.145038,26,46,25,25
3,101,2019-01-28,1.30,96,0.69,1.30,0.69,Male,173.0,53,3.610061,97.150104,36.010470,36.010470,98.816157,53.076923,26,46,26,26
4,101,2019-01-29,1.28,98,0.60,1.28,0.60,Male,173.0,53,3.610061,97.150104,35.456463,35.456463,100.874827,46.875000,25,48,23,23


In [4]:
def get_day_to_day_ecFEV1(df_for_ID, n_days, fev1_col="ecFEV1"):
    df_for_ID = df_for_ID.sort_values(by="Date Recorded", ascending=True)
    df_for_ID[f"day_to_day_{fev1_col}"] = df_for_ID[fev1_col].diff(periods=n_days)
    return df_for_ID

In [5]:
fev1_col = "ecFEV1"
fev1_col = "FEV1"
n_days = 1
df_d2d = df.groupby("ID").apply(get_day_to_day_ecFEV1, n_days, fev1_col)
d2d = df_d2d[f"day_to_day_{fev1_col}"].dropna().reset_index().drop(columns=["level_1"])

# Plot histogram of
fig = go.Figure()
xbins = dict(start=-0.505, end=0.5, size=0.01)
fig.add_trace(
    go.Histogram(x=d2d[f"day_to_day_{fev1_col}"], xbins=xbins, histnorm="probability")
)

# weight = 0.7
# Add gaussian with std as d2d
# mu = d2d["day_to_day_ecFEV1"].mean()
# std = d2d["day_to_day_ecFEV1"].std()
# print(f"mu: {mu}, std: {std}")
# x = np.linspace(-0.5, 0.5, 100)
# y1 = norm.pdf(x, mu, std*1.1)
# y1 = (1-weight) * y1 / y1.sum()
# fig.add_trace(go.Scatter(x=x, y=y1, mode='lines', name="Gaussian", line=dict(color='red')))
# # Add laplace
# y2 = norm.pdf(x, mu, 0.05)
# y2 = laplace.pdf(x, mu, 0.05)
# y2 = weight * y2 / y2.sum()
# fig.add_trace(go.Scatter(x=x, y=y2, mode='lines', name="Laplace", line=dict(color='green')))

# y = y1 + y2
# fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name="Combined", line=dict(color='blue')))

fig.update_xaxes(title_text=f"Day-to-day change in {fev1_col}")
fig.update_layout(title=f"Change in {fev1_col} over {n_days} days")
fig.show()

In [6]:
# Plot this per individual
for id in d2d["ID"].unique():
    fig = go.Figure()
    fig.add_trace(
        go.Histogram(
            x=d2d[d2d["ID"] == id]["day_to_day_ecFEV1"],
            xbins=xbins,
            histnorm="probability",
        )
    )
    fig.update_xaxes(title_text="Day-to-day change in ecFEV1", range=[-0.5, 0.5])
    title = f"ID {id} - Change in ecFEV1 over {n_days} days"
    fig.update_layout(title=title, width=800, height=400)
    fig.write_image(
        f"{dh.get_path_to_main()}/PlotsBreathe/Interconnecting_ARs_entries/day_to_day_ecFEV1/{title}.pdf"
    )


KeyboardInterrupt



In [15]:
# Quantify amount of same day to day values

fev1_col = "FEV1"
n_days = 1
df_d2d = df.groupby("ID").apply(get_day_to_day_ecFEV1, n_days, fev1_col)
d2d = df_d2d[f"day_to_day_{fev1_col}"].dropna().reset_index().drop(columns=["level_1"])
n_0_diff = (
    d2d[f"day_to_day_{fev1_col}"].where(d2d[f"day_to_day_{fev1_col}"] == 0).count()
)
print(f"Amount of 0 diff for FEV1 {n_0_diff}/{len(d2d)}")

fev1_col = "ecFEV1"
n_days = 1
df_d2d = df.groupby("ID").apply(get_day_to_day_ecFEV1, n_days, fev1_col)
d2d = df_d2d[f"day_to_day_{fev1_col}"].dropna().reset_index().drop(columns=["level_1"])
n_0_diff = (
    d2d[f"day_to_day_{fev1_col}"].where(d2d[f"day_to_day_{fev1_col}"] == 0).count()
)
print(f"Amount of 0 diff for ecFEV1 {n_0_diff}")

Amount of 0 diff for FEV1 2318/40908
Amount of 0 diff for ecFEV1 3030


In [54]:
df_valc = pd.DataFrame(d2d.value_counts()).reset_index().set_index('ID')
df_valc = df_valc[df_valc.index == '101']

# Bar plot of the number of days for each ID
fig = make_subplots(rows=1, cols=1)
fig.add_trace(
    go.Bar(y=df_valc.index, x=df_valc["count"], orientation="h", name="all data"), row=1, col=1
)
for val in df_valc["day_to_day_ecFEV1"].unique():
    fig.add_trace(go.Bar(y=df_valc.index, x=df_valc[df_valc["day_to_day_ecFEV1"] == val]["count"], orientation="h", name=f"diff={val}"), row=1, col=1)
# fig.add_trace(
#     go.Bar(y=df_valc.index, x=df_valc[df_valc["day_to_day_ecFEV1"] == 0]["count"], orientation="h", name="all data", marker_color=dh.get_blind_colours()[1]), row=1, col=1
# )
# fig.add_trace(
#     go.Bar(y=df_valc.index, x=df_valc[df_valc["day_to_day_ecFEV1"] == 0.01]["count"], orientation="h", name="all data", marker_color=dh.get_blind_colours()[2]), row=1, col=1
# )
fig.update_layout(title_text="Number of consecutive days for each ID", height=400, font_size=8)

fig.show()

In [62]:
import plotly.graph_objects as go

# Assuming df_valc is a DataFrame with columns: "count", "day_to_day_ecFEV1", and an index
# Create a list of unique values for day_to_day_ecFEV1
unique_values = df_valc["day_to_day_ecFEV1"].unique()

# Create a color map for each unique value
# color_map = {
#     value: color for value, color in zip(unique_values, ["red", "blue", "green", "orange", "purple"])
# }

# Create a figure
fig = go.Figure()

# Add a bar trace for each unique value of day_to_day_ecFEV1
for value in unique_values:
    filtered_data = df_valc[df_valc["day_to_day_ecFEV1"] == value]
    fig.add_trace(
        go.Bar(
            y=filtered_data.index,
            x=filtered_data["count"],
            orientation="h",
            name=f"day_to_day_ecFEV1: {value}",
            # marker=dict(color=color_map[value])
        )
    )

# Update layout to show the legend
fig.update_layout(
    barmode="stack",  # Stack the bars
    title="Bar Plot with Segments and Legend",
    xaxis_title="Count",
    yaxis_title="Index",
    legend_title="day_to_day_ecFEV1",
    height=300,
    width=1000
)

# Show the figure
fig.show()

## Day to day diff in L of FEV1

Knowing that 0.1 diff is approx 2%

In [16]:
df

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,ecFEV1,ecFEF2575,Sex,Height,Age,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy,ecFEF2575%ecFEV1,idx ecFEV1 (L),idx O2 saturation (%),idx ecFEF2575%ecFEV1,idx ecFEF25-75 % ecFEV1 (%)
0,101,2019-01-25,1.31,97,0.54,1.31,0.54,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,99.845492,41.221374,26,47,20,20
1,101,2019-01-26,1.31,98,0.57,1.31,0.57,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,100.874827,43.511450,26,48,21,21
2,101,2019-01-27,1.31,96,0.67,1.31,0.67,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,98.816157,51.145038,26,46,25,25
3,101,2019-01-28,1.30,96,0.69,1.30,0.69,Male,173.0,53,3.610061,97.150104,36.010470,36.010470,98.816157,53.076923,26,46,26,26
4,101,2019-01-29,1.28,98,0.60,1.28,0.60,Male,173.0,53,3.610061,97.150104,35.456463,35.456463,100.874827,46.875000,25,48,23,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41255,553,2023-10-08,4.12,98,4.20,4.12,4.20,Male,179.0,18,4.619116,97.042483,89.194549,89.194549,100.986699,101.941748,82,48,50,50
41256,553,2023-10-11,4.15,97,4.06,4.15,4.06,Male,179.0,18,4.619116,97.042483,89.844024,89.844024,99.956223,97.831325,83,47,48,48
41257,553,2023-11-06,4.13,97,4.84,4.13,4.84,Male,179.0,18,4.619116,97.042483,89.411041,89.411041,99.956223,117.191283,82,47,58,58
41258,553,2023-11-08,4.05,96,4.71,4.05,4.71,Male,179.0,18,4.619116,97.042483,87.679108,87.679108,98.925746,116.296296,81,46,58,58
